Framework for making a input catalog...

1) read in a csv file of objects

2) call tess_ephem for each object to see if it has been observed by TESS

3) if it has, summarize the TESS info

4) output two files...  one lists all the bodies searched, the other is a catalog of the objects observed by TESS

In [1]:
from tess_ephem import ephem
import numpy as np
from astropy.time import Time
import pandas as pd
from itertools import repeat
from datetime import datetime

Starting with a list made by doing a search in Horizons small body search interface  (https://ssd.jpl.nasa.gov/sbdb_query.cgi). I don't see any documentation for an API, so you have to select the outputs manually.  To start, I've got the following columns:  PDE, full_name, a, e, i om, w, q, ad, per_y, condition_code, H, rot_per, class.  Whether or not these are the right columns is totally tbd!  Note, currently as long as the input list as a column for primary designation (pdes), the code will work.  The output catalog will just copy over any additional input columns into the output catalog

The current catalog returns a line for every unique combination of asteroid and sector.  (In otherwords, if an asteroid is present in multiple sectors, it will have multiple lines.)

Things to check -- do we need to do the same for cameras?  (Right now the code largely assumes that there is only one camera involved, but it does output the number of cameras (and ccds), so we can check this assumption and revisit later on.

In [2]:
#edit this cell to define the names of the input and output files
input_list = "shortishlist.csv"  #small input list for test / demos
output_list = "asteroid_list.csv"
catalog_name = "catalog_short.csv"

In [3]:
# Want to see how long this all takes...
now = datetime.now().time() # time object
print("now =", now)

now = 10:39:08.802296


In [4]:
asteroid_list = pd.read_csv(input_list) 
num_asteroids = asteroid_list.shape[0]
num_asteroids

99

In [5]:
asteroid_list.columns

Index(['pdes', 'full_name', 'a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y',
       'condition_code', 'class', 'H', 'H_sigma', 'G', 'diameter',
       'diameter_sigma', 'extent', 'albedo', 'rot_per', 'spec_B', 'spec_T'],
      dtype='object')

In [6]:
# first off, a function that takes in tess_ephem output, asteroid info, and returns a one row data frame with results

def make_catalog_entry(ephem, asteroid):
    #calc values of interest
    num_of_days = ephem.shape[0]
    asteroid["num_of_days"] = num_of_days
    asteroid["sector"] = ephem.sector[0]
    asteroid["num_cameras"] = ephem.camera.nunique()
    asteroid["camera"] = ephem.camera[0]
    asteroid["num_ccd"] = ephem.ccd.nunique()
    asteroid["first_time"] = Time(ephem.index).jd[0] - 2457000  #constant is to conver to "TESS JD"
    asteroid["first_ccd"] = ephem.ccd[0]
    asteroid["first_column"] = ephem.column[0]
    asteroid["first_row"] = ephem.row[0]
    asteroid["last_time"] = Time(ephem.index).jd[-1] - 2457000
    asteroid["last_ccd"] = ephem.ccd[-1]
    asteroid["last_column"]  = ephem.column[(num_of_days-1)]
    asteroid["last_row"]= ephem.row[(num_of_days-1)]
    asteroid["max_pix_per_hour"] = ephem.pixels_per_hour.max()
    asteroid["min_Vmag"] = ephem.vmag.min()
    asteroid["mean_Vmag"] = ephem.vmag.mean()
    asteroid["max_Vmag"] = ephem.vmag.max()
    asteroid["min_phase_angle"] = ephem.phase_angle.min()
    asteroid["mean_phase_angle"] = ephem.phase_angle.mean()
    asteroid["max_phase_angle"] = ephem.phase_angle.max()
    asteroid["min_sun_distance"] = ephem.sun_distance.min()
    asteroid["mean_sun_distance"] = ephem.sun_distance.mean()
    asteroid["max_sun_distance"] = ephem.sun_distance.max()
    asteroid["min_tess_distance"] = ephem.obs_distance.min()
    asteroid["mean_tess_distance"] = ephem.obs_distance.mean()
    asteroid["max_tess_distance"] = ephem.obs_distance.max()
    
    return asteroid

there are two dataframes modified / created by this loop...

the original asteroid list will have a column added for whether or not the object is seen by tess.  this data frame includes a row for every asteroid we query about.

the output catalog will have an entry for each unique combination of asteroid, sector for every object seen by tess.  this data frame only has rows for objects seen by tess.

In [7]:
#now a version that results in multiple entries if there are multiple sectors
asteroid_catalog = []

false_list = []
false_list.extend(repeat(False, num_asteroids))
asteroid_list["in_Tess"] = false_list

starting_row = 0 #set to zero for standard use.  set to other values to restart...

for idx, ad in asteroid_list[starting_row:].iterrows(): 
    try:
        tt = ephem(str(ad["pdes"]), verbose=True)
    except:  #yes, this is hacky...  but sometimes the network just needs a minute to recover
        sleep(60)
        tt = ephem(str(ad["pdes"]), verbose=True)
    finally:
        #print(asteroid["pdes"], tt.shape[0])
        if tt.shape[0] == 0:
            continue
        #ad["in_Tess"] = True  #dunno why, but this doesn't modify the value in asteroid_list
        asteroid_list.loc[idx,"in_Tess"] = True
        #this_asteroid = ad.copy() #this works, but only if i use the ad["in_Tess"] = True
        this_asteroid = asteroid_list.loc[idx].copy()
        this_asteroid["num_sectors"] = tt.sector.nunique()
        for sector in tt.sector.unique():
            this_sector = tt.loc[tt["sector"] == sector]
            new_entry = make_catalog_entry(this_sector, this_asteroid.copy())
            asteroid_catalog.append(new_entry)
    
                            
asteroid_catalog = pd.DataFrame(asteroid_catalog)

In [8]:
# Want to see how long this all takes...
now = datetime.now().time() # time object
print("now =", now)

now = 10:39:46.271857


In [9]:
asteroid_catalog.head()


,pdes,full_name,a,e,i,om,w,q,ad,per_y,...,max_Vmag,min_phase_angle,mean_phase_angle,max_phase_angle,min_sun_distance,mean_sun_distance,max_sun_distance,min_tess_distance,mean_tess_distance,max_tess_distance
2,3,3 Juno (A804 RA),2.668285,0.256936,12.991043,169.851484,248.066191,1.982706,3.353865,4.358696,...,7.538,11.3189,11.879536,13.0354,1.983404,1.983570,1.983950,1.034468,1.040812,1.053196
2,3,3 Juno (A804 RA),2.668285,0.256936,12.991043,169.851484,248.066191,1.982706,3.353865,4.358696,...,9.739,2.5352,3.953108,5.9178,3.028313,3.052795,3.079113,2.056427,2.069661,2.104232
5,6,6 Hebe (A847 NA),2.424533,0.203219,14.739653,138.643432,239.736274,1.931822,2.917243,3.775290,...,8.637,7.8225,8.842506,10.7259,2.194220,2.216477,2.240871,1.255080,1.266058,1.291871
5,6,6 Hebe (A847 NA),2.424533,0.203219,14.739653,138.643432,239.736274,1.931822,2.917243,3.775290,...,10.080,6.2696,6.599063,7.6863,2.915509,2.916413,2.917145,1.951485,1.954525,1.965591
6,7,7 Iris (A847 PA),2.387375,0.230145,5.521598,259.563941,145.201546,1.837933,2.936818,3.688835,...,9.785,2.7065,4.304248,7.4472,2.811142,2.828344,2.844738,1.827543,1.843691,1.884886


In [10]:
asteroid_list.head()

,pdes,full_name,a,e,i,om,w,q,ad,per_y,...,H_sigma,G,diameter,diameter_sigma,extent,albedo,rot_per,spec_B,spec_T,in_Tess
0,1,1 Ceres (A801 AA),2.769165,0.076009,10.594067,80.305531,73.597695,2.558684,2.979647,4.608202,...,NaN,0.12,939.400,0.200,964.4 x 964.2 x 891.8,0.0900,9.074170,C,G,False
1,2,2 Pallas (A802 FA),2.773841,0.229972,34.832931,173.024741,310.202392,2.135935,3.411748,4.619880,...,NaN,0.11,545.000,18.000,582x556x500,0.1010,7.813200,B,B,False
2,3,3 Juno (A804 RA),2.668285,0.256936,12.991043,169.851484,248.066191,1.982706,3.353865,4.358696,...,NaN,0.32,246.596,10.594,NaN,0.2140,7.210000,Sk,S,True
3,4,4 Vesta (A807 FA),2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,NaN,0.32,525.400,0.200,572.6 x 557.2 x 446.4,0.4228,5.342128,V,V,False
4,5,5 Astraea (A845 XA),2.574037,0.190913,5.367427,141.571025,358.648419,2.082619,3.065455,4.129814,...,NaN,NaN,106.699,3.140,NaN,0.2740,16.806000,S,S,False


In [11]:
asteroid_catalog.to_csv(catalog_name, index=False)

In [12]:
asteroid_list.to_csv(output_list,index=False)